This notebook was used during development to incrementally test the program for task 2

### Library

In [1]:
import psycopg2
import time
import itertools

### Constants

In [2]:
TABLE = 'task_1'

### Helper Methods

In [3]:
def create_query_string(LHS=[], RHS=[], table=TABLE):
    "creates a the query that will return whether RHS has a functional dependency on LHS."
    if type(LHS) == str : 
        LHS = [LHS]
    if type(RHS) == str : 
        RHS = [RHS]
    
    
    LHS_string = ''
    for lhs in LHS:
        LHS_string += lhs + ', '
    LHS_string = LHS_string[:-2]

    RHS_string_1 = ''
    RHS_string_2 = ''
    RHS_string_3 = ''
    for rhs in RHS:
        RHS_string_1 += f'COUNT({rhs}), '
        RHS_string_2 += rhs + ', '
        RHS_string_3 += f'COUNT({rhs}) > 1 AND '
    
    RHS_string_1 = RHS_string_1[:-2]
    RHS_string_2 = RHS_string_2[:-2]
    RHS_string_3 = RHS_string_3[:-5]

    query_string = f'SELECT EXISTS (SELECT {LHS_string}, {RHS_string_1} FROM (SELECT DISTINCT {LHS_string}, {RHS_string_2} FROM {table}) AS SQ GROUP BY {LHS_string} HAVING {RHS_string_3});'
    # print("Query: ", query_string)
    return query_string


def find_fds (LHS, RHS, cur, conn, table=TABLE) :
    start_time = time.time()
    cur.execute(create_query_string(LHS=LHS, RHS=RHS, table=table))
    conn.commit()
    duration = round(time.time() - start_time,3)
    # result = cur.fetchall()[0][0]
    result = cur.fetchall()
    # print(f"RESULT: {result}")
    if (result[0][0]):
        print(f"Functional Dependency ABSENT in : {LHS} -> {RHS}, Duration: {duration}")
    else:
        print(f"Functional Dependency PRESENT in : {LHS} -> {RHS}, Duration: {duration}")

### PSQL

#### Connection

In [4]:
conn = psycopg2.connect("dbname=csci_620_ass_2")
print("Database connected successfully")
cur = conn.cursor()
conn.commit()

Database connected successfully


#### Get Attributes

In [5]:
cur.execute(f"Select * FROM {TABLE} LIMIT 0")
attributes = [desc[0] for desc in cur.description]

### All possible combinations

In [6]:
for RHS_n_LHS_quantity in range(2, len(attributes) + 1):
    for subset in itertools.combinations(attributes, RHS_n_LHS_quantity):
        RHS_LHS = list(subset)
        RHS = RHS_LHS[0]
        LHS = RHS_LHS[1:]
        # print(f"Investigating: {LHS} -> {RHS}")
        find_fds(LHS=LHS, RHS=RHS, cur=cur, conn=conn)

Investigating: ['type'] -> movieid
Functional Dependency ABSENT in : ['type'] -> movieid, Duration: 2.775
Investigating: ['startyear'] -> movieid
Functional Dependency ABSENT in : ['startyear'] -> movieid, Duration: 2.539
Investigating: ['runtime'] -> movieid
Functional Dependency ABSENT in : ['runtime'] -> movieid, Duration: 2.717
Investigating: ['avgrating'] -> movieid
Functional Dependency ABSENT in : ['avgrating'] -> movieid, Duration: 3.983
Investigating: ['genreid'] -> movieid
Functional Dependency ABSENT in : ['genreid'] -> movieid, Duration: 3.869
Investigating: ['genre'] -> movieid
Functional Dependency ABSENT in : ['genre'] -> movieid, Duration: 5.538
Investigating: ['memberid'] -> movieid
Functional Dependency ABSENT in : ['memberid'] -> movieid, Duration: 5.989
Investigating: ['birthyear'] -> movieid
Functional Dependency ABSENT in : ['birthyear'] -> movieid, Duration: 3.053
Investigating: ['character'] -> movieid
Functional Dependency ABSENT in : ['character'] -> movieid, 